In [121]:
# Start by importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
from typing import Optional, Union
from datetime import date
import time
pd.set_option('display.max_columns', 225)
import math
import os


from pybaseball import schedule_and_record
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import team_batting
from pybaseball import team_pitching
from pybaseball import batting_stats_range
from pybaseball import pitching_stats_range
from pybaseball import statcast_single_game
from pybaseball import playerid_reverse_lookup
from pybaseball import cache

In [120]:
from datetime import date, timedelta, datetime
import json
import os
import requests

today = date.today()
tomorrow = today + timedelta(days=1)

url1 = "https://api.the-odds-api.com/v4/sports/baseball_mlb/odds/?apiKey=b61a17f9cf62faf0bf64cc7e180cddc7&regions=us&markets=h2h"
url2 = "https://api.the-odds-api.com/v4/sports/baseball_mlb/odds/?apiKey=ff54285791dd7b39f67c27f0735741ec&regions=us&markets=h2h"

response = requests.get(url1)
if response.status_code == 429:
    print("API rate limit reached, switching to the second API key.")
    response = requests.get(url2)

# Check if the response indicates a rate limit or quota error.
# This example assumes the API returns a 429 status code when you've hit the limit.
# The exact status code or error message may vary depending on the API.
if response.status_code == 429:
    print("API rate limit reached, switching to the second API key.")
    response = make_request(api_key2)
    
team_dict = {
    'Diamondbacks': ['Arizona', 'Maj-NL'],
    'Braves': ['Atlanta', 'Maj-NL'],
    'Orioles': ['Baltimore', 'Maj-AL'],
    'Red Sox': ['Boston', 'Maj-AL'],
    'Cubs': ['Chicago', 'Maj-NL'],
    'White Sox': ['Chicago', 'Maj-AL'],
    'Reds': ['Cincinnati', 'Maj-NL'],
    'Guardians': ['Cleveland', 'Maj-AL'],
    'Rockies': ['Colorado', 'Maj-NL'],
    'Tigers': ['Detroit', 'Maj-AL'],
    'Astros': ['Houston', 'Maj-AL'],
    'Royals': ['Kansas City', 'Maj-AL'],
    'Angels': ['Los Angeles', 'Maj-AL'],
    'Dodgers': ['Los Angeles', 'Maj-NL'],
    'Marlins': ['Miami', 'Maj-NL'],
    'Brewers': ['Milwaukee', 'Maj-NL'],
    'Twins': ['Minnesota', 'Maj-AL'],
    'Mets': ['New York', 'Maj-NL'],
    'Yankees': ['New York', 'Maj-AL'],
    'Athletics': ['Oakland', 'Maj-AL'],
    'Phillies': ['Philadelphia', 'Maj-NL'],
    'Pirates': ['Pittsburgh', 'Maj-NL'],
    'Padres': ['San Diego', 'Maj-NL'],
    'Giants': ['San Francisco', 'Maj-NL'],
    'Mariners': ['Seattle', 'Maj-AL'],
    'Cardinals': ['St. Louis', 'Maj-NL'],
    'Rays': ['Tampa Bay', 'Maj-AL'],
    'Rangers': ['Texas', 'Maj-AL'],
    'Blue Jays': ['Toronto', 'Maj-AL'],
    'Nationals': ['Washington', 'Maj-NL']
}

def is_last_update_before_commence_time(last_update, commence_time):
    last_update_dt = datetime.strptime(last_update, "%Y-%m-%dT%H:%M:%SZ")
    commence_time_dt = datetime.strptime(commence_time, "%Y-%m-%dT%H:%M:%SZ")

    return last_update_dt < commence_time_dt

def get_american_odds(decimal_odds):
    if decimal_odds == 1:
        return 0.0
    if decimal_odds >= 2:
        return (decimal_odds - 1) * 100
    else:
        return -100 / (decimal_odds - 1)

def save_odds_data(game_key, home_team, away_team, bookmaker_key, home_team_odds, away_team_odds, input_string, save_daily_draftkings=False):
    if save_daily_draftkings:
        data_file = f"odds_history_{input_string}.json"
    else:
        data_file = "2023_season_odds.json"

    # Check if the file exists, otherwise create it with initial structure
    if not os.path.exists(data_file):
        with open(data_file, "w") as f:
            json.dump({}, f)

    # Load the existing data
    with open(data_file, "r") as f:
        data = json.load(f)

    # Update the data with the new odds
    if save_daily_draftkings:
        if game_key not in data:
            data[game_key] = {"home_team": home_team, "away_team": away_team, "odds_history": []}
        data[game_key]["odds_history"].append({"home_team_odds": home_team_odds, "away_team_odds": away_team_odds})
    else:
        game_key = f"{input_string}_{game_key}"
        if game_key not in data:
            data[game_key] = {"home_team": home_team, "away_team": away_team, "odds_history": {}}
        if bookmaker_key not in data[game_key]["odds_history"]:
            data[game_key]["odds_history"][bookmaker_key] = []
        data[game_key]["odds_history"][bookmaker_key].append({"home_team_odds": home_team_odds, "away_team_odds": away_team_odds})

    # Save the updated data back to the file
    with open(data_file, "w") as f:
        json.dump(data, f)

def get_team_name(full_team_name):
    for team, city_info in team_dict.items():
        city, _ = city_info
        combined_name = city + " " + team

        if combined_name == full_team_name:
            return team
    return full_team_name

if response.status_code == 200:
    # get the list of json objects from the response
    json_list = response.json()

    # get the input string from user
    input_string = today

    # filter the list of json objects by the "commence_time" key
    filtered_list = [obj for obj in json_list if input_string in obj.get("commence_time", "")]

    # iterate through the filtered list of json objects and extract the required fields
    for obj in filtered_list:
        # get the home_team and away_team fields from the base layer of the object
        home_team = obj.get("home_team", "")
        away_team = obj.get("away_team", "")

        # Get the team names without the city
        home_team_name = get_team_name(home_team)

        away_team_name = get_team_name(away_team)
        bookmakers_list = obj.get("bookmakers", [])
        all_bookmakers_odds = {}
        for bookmaker in bookmakers_list:
            bookmaker_key = bookmaker.get("key")
            last_update = bookmaker.get("last_update", "")
            commence_time = obj.get("commence_time", "")
            if not is_last_update_before_commence_time(last_update, commence_time):
                continue
            markets_list = bookmaker.get("markets", [])
            if len(markets_list) > 0:
                outcomes_list = markets_list[0].get("outcomes", [])
                home_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == home_team), None)
                away_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == away_team), None)

                home_team_odds = get_american_odds(home_team_outcome.get("price") if home_team_outcome is not None else "")
                away_team_odds = get_american_odds(away_team_outcome.get("price") if away_team_outcome is not None else "")

                # Save the extracted fields for this object
                game_key = home_team_name + "_vs_" + away_team_name

                # Save daily DraftKings data
                if bookmaker_key == "draftkings":
                    save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, home_team_odds, away_team_odds, input_string, save_daily_draftkings=True)

                # Collect season odds data for all sportsbooks
                all_bookmakers_odds[bookmaker_key] = {"home_team_odds": home_team_odds, "away_team_odds": away_team_odds}

        # Save season odds data for all sportsbooks
        for bookmaker_key, odds in all_bookmakers_odds.items():
            save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, odds["home_team_odds"], odds["away_team_odds"], input_string)

else:
    print("Error: Failed to fetch data from the API.")

if response.status_code == 200:
    # get the list of json objects from the response
    json_list = response.json()

    # get the input string from user
    input_string = tomorrow

    # filter the list of json objects by the "commence_time" key
    filtered_list = [obj for obj in json_list if input_string in obj.get("commence_time", "")]

    # iterate through the filtered list of json objects and extract the required fields
    for obj in filtered_list:
        # get the home_team and away_team fields from the base layer of the object
        home_team = obj.get("home_team", "")
        away_team = obj.get("away_team", "")

        # Get the team names without the city
        home_team_name = get_team_name(home_team)

        away_team_name = get_team_name(away_team)
        bookmakers_list = obj.get("bookmakers", [])
        all_bookmakers_odds = {}
        for bookmaker in bookmakers_list:
            bookmaker_key = bookmaker.get("key")
            markets_list = bookmaker.get("markets", [])
            if len(markets_list) > 0:
                outcomes_list = markets_list[0].get("outcomes", [])
                home_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == home_team), None)
                away_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == away_team), None)

                home_team_odds = get_american_odds(home_team_outcome.get("price") if home_team_outcome is not None else "")
                away_team_odds = get_american_odds(away_team_outcome.get("price") if away_team_outcome is not None else "")

                # Save the extracted fields for this object
                game_key = home_team_name + "_vs_" + away_team_name

                # Save daily DraftKings data
                if bookmaker_key == "draftkings":
                    save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, home_team_odds, away_team_odds, input_string, save_daily_draftkings=True)

                # Collect season odds data for all sportsbooks
                all_bookmakers_odds[bookmaker_key] = {"home_team_odds": home_team_odds, "away_team_odds": away_team_odds}

        # Save season odds data for all sportsbooks
        for bookmaker_key, odds in all_bookmakers_odds.items():
            save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, odds["home_team_odds"], odds["away_team_odds"], input_string)

else:
    print("Error: Failed to fetch data from the API.")    


Today's date: 2023-04-17
Tomorrow's date: 2023-04-18


In [105]:
team_dict = {
    'Diamondbacks': ['Arizona', 'Maj-NL'],
    'Braves': ['Atlanta', 'Maj-NL'],
    'Orioles': ['Baltimore', 'Maj-AL'],
    'Red Sox': ['Boston', 'Maj-AL'],
    'Cubs': ['Chicago', 'Maj-NL'],
    'White Sox': ['Chicago', 'Maj-AL'],
    'Reds': ['Cincinnati', 'Maj-NL'],
    'Guardians': ['Cleveland', 'Maj-AL'],
    'Rockies': ['Colorado', 'Maj-NL'],
    'Tigers': ['Detroit', 'Maj-AL'],
    'Astros': ['Houston', 'Maj-AL'],
    'Royals': ['Kansas City', 'Maj-AL'],
    'Angels': ['Los Angeles', 'Maj-AL'],
    'Dodgers': ['Los Angeles', 'Maj-NL'],
    'Marlins': ['Miami', 'Maj-NL'],
    'Brewers': ['Milwaukee', 'Maj-NL'],
    'Twins': ['Minnesota', 'Maj-AL'],
    'Mets': ['New York', 'Maj-NL'],
    'Yankees': ['New York', 'Maj-AL'],
    'Athletics': ['Oakland', 'Maj-AL'],
    'Phillies': ['Philadelphia', 'Maj-NL'],
    'Pirates': ['Pittsburgh', 'Maj-NL'],
    'Padres': ['San Diego', 'Maj-NL'],
    'Giants': ['San Francisco', 'Maj-NL'],
    'Mariners': ['Seattle', 'Maj-AL'],
    'Cardinals': ['St. Louis', 'Maj-NL'],
    'Rays': ['Tampa Bay', 'Maj-AL'],
    'Rangers': ['Texas', 'Maj-AL'],
    'Blue Jays': ['Toronto', 'Maj-AL'],
    'Nationals': ['Washington', 'Maj-NL']
}

In [110]:
import requests

# make the GET request to the API and get the response
url = "https://api.the-odds-api.com/v4/sports/baseball_mlb/odds/?apiKey=b61a17f9cf62faf0bf64cc7e180cddc7&regions=us&markets=h2h"
response = requests.get(url)

In [111]:
import json
import os
import requests

def get_american_odds(decimal_odds):
    if decimal_odds == 1:
        return 0.0
    if decimal_odds >= 2:
        return (decimal_odds - 1) * 100
    else:
        return -100 / (decimal_odds - 1)

def save_odds_data(game_key, home_team, away_team, bookmaker_key, home_team_odds, away_team_odds, input_string, save_daily_draftkings=False):
    if save_daily_draftkings:
        data_file = f"odds_history_{input_string}.json"
    else:
        data_file = "2023_season_odds.json"

    # Check if the file exists, otherwise create it with initial structure
    if not os.path.exists(data_file):
        with open(data_file, "w") as f:
            json.dump({}, f)

    # Load the existing data
    with open(data_file, "r") as f:
        data = json.load(f)

    # Update the data with the new odds
    if save_daily_draftkings:
        if game_key not in data:
            data[game_key] = {"home_team": home_team, "away_team": away_team, "odds_history": []}
        data[game_key]["odds_history"].append({"home_team_odds": home_team_odds, "away_team_odds": away_team_odds})
    else:
        game_key = f"{input_string}_{game_key}"
        if game_key not in data:
            data[game_key] = {"home_team": home_team, "away_team": away_team, "odds_history": {}}
        if bookmaker_key not in data[game_key]["odds_history"]:
            data[game_key]["odds_history"][bookmaker_key] = []
        data[game_key]["odds_history"][bookmaker_key].append({"home_team_odds": home_team_odds, "away_team_odds": away_team_odds})

    # Save the updated data back to the file
    with open(data_file, "w") as f:
        json.dump(data, f)

def get_team_name(full_team_name):
    for team, city_info in team_dict.items():
        city, _ = city_info
        combined_name = city + " " + team

        if combined_name == full_team_name:
            return team
    return full_team_name

if response.status_code == 200:
    # get the list of json objects from the response
    json_list = response.json()

    # get the input string from user
    input_string = "2023-04-17"

    # filter the list of json objects by the "commence_time" key
    filtered_list = [obj for obj in json_list if input_string in obj.get("commence_time", "")]

    # iterate through the filtered list of json objects and extract the required fields
    for obj in filtered_list:
        # get the home_team and away_team fields from the base layer of the object
        home_team = obj.get("home_team", "")
        away_team = obj.get("away_team", "")

        # Get the team names without the city
        home_team_name = get_team_name(home_team)

        away_team_name = get_team_name(away_team)
        bookmakers_list = obj.get("bookmakers", [])
        all_bookmakers_odds = {}
        for bookmaker in bookmakers_list:
            bookmaker_key = bookmaker.get("key")
            markets_list = bookmaker.get("markets", [])
            if len(markets_list) > 0:
                outcomes_list = markets_list[0].get("outcomes", [])
                home_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == home_team), None)
                away_team_outcome = next((outcome for outcome in outcomes_list if outcome.get("name") == away_team), None)

                home_team_odds = get_american_odds(home_team_outcome.get("price") if home_team_outcome is not None else "")
                away_team_odds = get_american_odds(away_team_outcome.get("price") if away_team_outcome is not None else "")

                # Save the extracted fields for this object
                game_key = home_team_name + "_vs_" + away_team_name

                # Save daily DraftKings data
                if bookmaker_key == "draftkings":
                    save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, home_team_odds, away_team_odds, input_string, save_daily_draftkings=True)

                # Collect season odds data for all sportsbooks
                all_bookmakers_odds[bookmaker_key] = {"home_team_odds": home_team_odds, "away_team_odds": away_team_odds}

        # Save season odds data for all sportsbooks
        for bookmaker_key, odds in all_bookmakers_odds.items():
            save_odds_data(game_key, home_team_name, away_team_name, bookmaker_key, odds["home_team_odds"], odds["away_team_odds"], input_string)

else:
    print("Error: Failed to fetch data from the API.")

In [72]:
def get_first_last_odds(matchup_key, input_string):
    data_file = f"odds_history_{input_string}.json"

    # Check if the file exists
    if not os.path.exists(data_file):
        print("Error: The data file does not exist.")
        return 0, 0, 0, 0

    # Load the existing data
    with open(data_file, "r") as f:
        data = json.load(f)

    if matchup_key not in data:
        print("Error: The matchup key does not exist in the data file.")
        return 0, 0, 0, 0

    odds_history = data[matchup_key]["odds_history"]
    if not odds_history:
        print("Error: There is no odds history for this matchup.")
        return 0, 0, 0, 0

    first_home_odds = odds_history[0]["home_team_odds"]
    last_home_odds = odds_history[-1]["home_team_odds"]
    first_away_odds = odds_history[0]["away_team_odds"]
    last_away_odds = odds_history[-1]["away_team_odds"]

    return first_home_odds, last_home_odds, first_away_odds, last_away_odds
first_home_odds, last_home_odds, first_away_odds, last_away_odds = get_first_last_odds("Twins_vs_White Sox", "2023-04-10")
print(first_home_odds, last_home_odds, first_away_odds, last_away_odds)

Error: The data file does not exist.
0 0 0 0


In [57]:
df = pd.read_csv('razzball.csv')

# set the 'Name' column as the index
df = df.set_index('Name')

# convert the dataframe to a dictionary with the 'MLBAMID' column as values
mlbamids = df['MLBAMID'].to_dict()


In [132]:
import requests
from bs4 import BeautifulSoup

def get_mlb_matchups_and_pitchers(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch URL: {url}. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    matchups_divs = soup.find_all('div', class_='starting-lineups__matchup')

    matchups_and_pitchers = []
    for matchup_div in matchups_divs:
        team_names_div = matchup_div.find('div', class_='starting-lineups__team-names')
        home_team_span = team_names_div.find('span', class_='starting-lineups__team-name--home')
        away_team_span = team_names_div.find('span', class_='starting-lineups__team-name--away')
        away_team = away_team_span.text.strip()
        if away_team == "D-backs":
            away_team = "Diamondbacks"
        home_team = home_team_span.text.strip()
        if home_team == "D-backs":
            home_team = "Diamondbacks"

        pitcher_names_div = matchup_div.find('div', class_='starting-lineups__pitcher-overview')
        pitcher_anchors = pitcher_names_div.find_all('a', class_='starting-lineups__pitcher--link')
        if len(pitcher_anchors) < 4:
            continue
        away_pitcher = pitcher_anchors[1].text.strip()
        home_pitcher = pitcher_anchors[3].text.strip()

        matchup_and_pitchers = {
            'home_team': home_team,
            'away_team': away_team,
            'home_pitcher': mlbamids[home_pitcher],
            'away_pitcher': mlbamids[away_pitcher]
        }
        matchups_and_pitchers.append(matchup_and_pitchers)

    return matchups_and_pitchers

url = 'https://www.mlb.com/starting-lineups/2023-04-20'
matchups_and_pitchers = get_mlb_matchups_and_pitchers(url)

# for idx, matchup_and_pitchers in enumerate(matchups_and_pitchers, start=1):
#     print(f"Matchup {idx}: {matchup_and_pitchers['away_team']} vs {matchup_and_pitchers['home_team']}")
#     print(f"Away pitcher: {matchup_and_pitchers['away_pitcher']}")
#     print(f"Home pitcher: {matchup_and_pitchers['home_pitcher']}")
#     print()

In [59]:
def team_abreviator(team,league):
    if team=='Atlanta':
        return 'ATL'
    elif team=='Arizona':
        return 'ARI'
    elif team=='Baltimore':
        return 'BAL'
    elif team=='Boston':
        return 'BOS'
    elif team=='Chicago':
        if league=='MLB-AL':
            return 'CHW'
        else:
            return 'CHC'
    elif team=='Cincinnati':
        return 'CIN'
    elif team=='Cleveland':
        return 'CLE'
    elif team=='Colorado':
        return 'COL'
    elif team=='Detroit':
        return 'DET'
    elif team=='Houston':
        return 'HOU'
    elif team=='Kansas City':
        return 'KC'
    elif team=='Los Angeles':
        if league=='MLB-AL':
            return 'LAA'
        else:
            return 'LAD'
    elif team=='Minnesota':
        return 'MIN'
    elif team=='Milwaukee':
        return 'MIL'
    elif team=='Miami':
        return 'MIA'
    elif team=='New York':
        if league=='MLB-AL':
            return 'NYY'
        else:
            return 'NYM'
    elif team=='Oakland':
        return 'OAK'
    elif team=='Pittsburgh':
        return 'PIT'
    elif team=='Philadelphia':
        return 'PHI'
    elif team=='San Diego':
        return 'SD'
    elif team=='San Francisco':
        return 'SF'
    elif team=='Seattle':
        return 'SEA'
    elif team=='St. Louis':
        return 'STL'
    elif team=='Texas':
        return 'TEX'
    elif team=='Tampa Bay':
        return 'TB'
    elif team=='Toronto':
        return 'TOR'
    elif team=='Washington':
        return 'WSN'
    else:
        print('Team not available in every pitch data')
        return 0

In [60]:
def get_record_stats(date, lookback_days, team_abr):
    lookback_end = str(pd.to_datetime(date)-pd.to_timedelta(1,unit='D'))[:10]
    lookback_start = str(pd.to_datetime(date)-pd.to_timedelta(lookback_days+1,unit='D'))[:10]
    
    time.sleep(5)
    record = schedule_and_record(2023, team_abr)
    record.set_index('Date',inplace=True)
    
    y = pd.to_datetime(date)
    z = y.day
    date_string = str(y.strftime('%A, %b '))+str(z)
    
    y2 = pd.to_datetime(lookback_end)
    z2 = y2.day
    date_string2 = str(y2.strftime('%A, %b '))+str(z2)
    
    lookback_pushback = 0
    
#     y,y2,y3=pd.to_datetime(date),pd.to_datetime(lookback_end),pd.to_datetime(lookback_start)
#     z,z2,z3=y.day,y2.day,y3.day
#     date_string,date_string2,date_string3=str(y.strftime('%A, %b '))+str(z),str(y2.strftime('%A, %b '))+str(z2),str(y3.strftime('%A, %b '))+str(z3)
    
    
    while date_string2 not in record.index:
        lookback_pushback += 1
        y2 -= pd.DateOffset(days=1)
        date_string2 = str(y2.strftime('%A, %b ')) + str(y2.day)
    
    if date_string2 in record.index:
        team_record = record.loc[date_string2]['W-L']
    else:
        team_record = '0-0'
    
#     team_record=record.loc[date_string2]['W-L']
    current_wins,current_losses=int(team_record.split('-')[0]),int(team_record.split('-')[1])
    pct=current_wins/(current_wins+current_losses)

    lookback_start = str(pd.to_datetime(date)-pd.to_timedelta(lookback_days+1+lookback_pushback,unit='D'))[:10]
    y3 = pd.to_datetime(lookback_start)
    z3 = y3.day
    date_string3 = str(y3.strftime('%A, %b '))+str(z3)
    
    while date_string3 not in record.index:
        y3 -= pd.DateOffset(days=1)
        date_string3 = str(y3.strftime('%A, %b ')) + str(y3.day)

    if date_string3 in record.index:
        record_lookback = record.loc[date_string3]['W-L']
    else:
        record_lookback = '0-0'
    lookback_wins,lookback_losses=int(record_lookback.split('-')[0]),int(record_lookback.split('-')[1])
    recent_wins,recent_losses=current_wins-lookback_wins,current_losses-lookback_losses  
    streak = recent_wins / (recent_wins + recent_losses)
    
    return pct, streak

In [133]:
start_date = '2023-04-19'
end_date = '2023-04-19'

pickle_file = "statcast_stats.pkl"

date_range = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
date_range = date_range.values
# # Prepare the new data
new_statcast_stats = statcast(start_date, end_date)

# Check if the file exists
if os.path.exists(pickle_file):
    # Load the existing DataFrame from the pickle file
    statcast_stats = pd.read_pickle(pickle_file)
    
    # Append new rows to the existing DataFrame
    statcast_stats = statcast_stats.append(new_statcast_stats, ignore_index=True)
    statcast_stats = statcast_stats.drop_duplicates()
    statcast_stats = statcast_stats.reset_index(drop=True)
    
else:
    # Create a new DataFrame with the new_data as the first rows
    statcast_stats = new_statcast_stats

# Save the updated DataFrame back to the pickle file
statcast_stats.to_pickle(pickle_file)

pickle_file_1 = "pitching_stats.pkl"

# Check if the file exists
if os.path.exists(pickle_file_1):
    # Load the existing DataFrame from the pickle file
    pitching_stats = pd.read_pickle(pickle_file_1)
    
    for i in range(len(date_range)):
        time.sleep(10)
        temp = pitching_stats_range(date_range[i])
        print(date_range[i])
        temp['Date'] = date_range[i]
        pitching_stats = pitching_stats.append(temp, ignore_index=True)
    pitching_stats = pitching_stats.drop_duplicates()
    pitching_stats = pitching_stats.reset_index(drop=True)
    
    
else:
    # Create a new DataFrame with the new_data as the first rows
    pitching_stats = pd.DataFrame()
    
    for i in range(len(date_range)):
        time.sleep(10)
        temp = pitching_stats_range(date_range[i])
        print(date_range[i])
        temp['Date'] = date_range[i]
        pitching_stats = pitching_stats.append(temp, ignore_index=True)
    pitching_stats = pitching_stats.drop_duplicates()
    pitching_stats = pitching_stats.reset_index(drop=True)

# Save the updated DataFrame back to the pickle file
pitching_stats.to_pickle(pickle_file_1)

pickle_file_2 = "batting_stats.pkl"

# Check if the file exists
if os.path.exists(pickle_file_2):
    # Load the existing DataFrame from the pickle file
    batting_stats = pd.read_pickle(pickle_file_2)
    
    for i in range(len(date_range)):
        time.sleep(10)
        temp = batting_stats_range(date_range[i])
        print(date_range[i])
        temp['Date'] = date_range[i]
        batting_stats = batting_stats.append(temp, ignore_index=True)
    batting_stats = batting_stats.drop_duplicates()
    batting_stats = batting_stats.reset_index(drop=True)
    
    
else:
    # Create a new DataFrame with the new_data as the first rows
    batting_stats = pd.DataFrame()
    
    for i in range(len(date_range)):
        time.sleep(10)
        temp = batting_stats_range(date_range[i])
        print(date_range[i])
        temp['Date'] = date_range[i]
        batting_stats = batting_stats.append(temp, ignore_index=True)
    batting_stats = batting_stats.drop_duplicates()
    batting_stats = batting_stats.reset_index(drop=True)

# Save the updated DataFrame back to the pickle file
batting_stats.to_pickle(pickle_file_2)

This is a large query, it may take a moment to complete


100%|█████████████████████████████████████████████| 1/1 [00:16<00:00, 16.51s/it]


2023-04-19
2023-04-19


In [134]:
# Protocol:
# 1. Get data by calling statcast for days from lookback_start to lookback_end
# 2. Input lookback_start and lookback_end as the beginning of the season to 
#    the day before current.

def get_all_starters(data,lookback_start,lookback_end):
    dates=pd.date_range(lookback_start,lookback_end)
    all_starters={}
    for day in dates:
        day_starters=[]
        day_data=data[data.game_date==day]
        today_games=day_data.game_pk.unique()
        for game in today_games:
            game_stats=day_data[day_data.game_pk==game]
            home_counter=0
            away_counter=0
            l=game_stats.pitcher.value_counts().keys()
            for pitcher in l:
                m=game_stats[game_stats.pitcher==pitcher]
                m.reset_index(drop=True, inplace=True)
                if home_counter==0 and m.inning_topbot[0]=='Bot':
                    home_starter_id=pitcher
                    home_counter+=1
                elif away_counter==0 and m.inning_topbot[0]=='Top':
                    away_starter_id=pitcher
                    away_counter=+1
                else: 
                    None
            day_starters.append(str(home_starter_id))
            day_starters.append(str(away_starter_id))
        exit_date=day.strftime('%Y-%m-%d')
        all_starters.update({exit_date:day_starters})
    return all_starters
starters_on_day=get_all_starters(statcast_stats,'2023-03-30','2023-04-19')

In [65]:
# Protocol:
# 1. Get starters_on_day by running get_all_relievers
# 2. Get data by pulling from pitching_stats_range for the lookback days (day before current back to X days before, which should be 8?)
# 3. Input lookback_start and lookback_end as the same lookback window used to source data
def get_all_relievers(starters_on_day,data,lookback_start,lookback_end):
    dates=pd.date_range(lookback_start,lookback_end)
    dates=dates.astype(str)
    all_starters=[]
    for day in dates:
        for player in starters_on_day[day]:
            all_starters.append(player)
    f=pd.DataFrame(all_starters,columns=['starters'])
    f=f.starters.unique()
    g=pd.DataFrame(f,columns=['starters'])
    h=pd.merge(data,g,how='outer',left_on='mlbID',right_on='starters',indicator=True)
    relievers=h[h._merge!='both']
    return data

In [66]:
reliever_columns=['team_relief','league_relief','innings','hits','bb',
                      'k','at_bats','doub','trip','hr','era','ba',
                     'slg','obp','est_ba_sa','est_woba_sa','sum_woba']
def recent_bullpen(data,every_pitch,team,league,lookback_start,lookback_end):
    pen=data[data.Tm==team]
    pen=pen[pen.Lev==league]
    pen.IP=((pen.IP-round(pen.IP,0))*(10/3))+(round(pen.IP,0))
    innings=pen.IP.sum()
    hits=pen.H.sum()*9/innings
    bb=(pen.BB.sum()+pen.HBP.sum()+pen.IBB.sum())*9/innings
    k=pen.SO.sum()*9/innings
    at_bats=pen.AB.sum()*9/innings
    doub=pen['2B'].sum()*9/innings
    trip=pen['3B'].sum()*9/innings
    hr=pen.HR.sum()*9/innings
    era=pen.ER.sum()/innings*9
    ba=hits/at_bats
    slg=round((hits+doub+(trip*2)+(hr*3))/at_bats,3)
    obp=round((hits+bb)/(at_bats+bb),3)
    data2=every_pitch[every_pitch.game_date<=lookback_end]
    data2=data2[data2.game_date>=lookback_start]
    team_abrev=team_abreviator(team,league)
    data3=data2[data2.home_team==team_abrev]
    data3=data3[data3.inning_topbot=='Bot']
    data4=data2[data2.away_team==team_abrev]
    data4=data4[data4.inning_topbot=='Top']
    data5=pd.concat([data3,data4])
    data5=data5.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
    est_ba_sa=data5.estimated_ba_using_speedangle.mean()
    est_woba_sa=data5.estimated_woba_using_speedangle.mean()
    sum_woba=data5.woba_value.sum()/innings                  
    df=pd.DataFrame([[team,league,innings,hits,bb,
                      k,at_bats,doub,trip,hr,era,ba,
                     slg,obp,est_ba_sa,est_woba_sa,sum_woba]],columns=reliever_columns)
    return df

In [67]:
batting_columns=['team','league','ab','runs','hits','doub','trip','hr','rbi','bb','avg','obp','slg',
                'est_ba_sa','est_woba_sa','sum_woba']

def recent_team_batting(data,every_pitch,start,end,team,league):
    data=data[data.loc[:, 'Tm'] == team]
    data=data[data.loc[:, 'Lev'] == league]
    ab=data.AB.sum()
    hits=data.H.sum()
    bb=data.BB.sum()+data.IBB.sum()+data.HBP.sum()
    doub=data['2B'].sum()
    trip=data['3B'].sum()
    hr=data.HR.sum()
    rbi=data.RBI.sum()
    avg=round(hits/ab,3)
    obp=round((hits+bb)/ab,3)
    slg=round((hits+doub+(trip*2)+(hr*3))/ab,3)
    runs=data.R.sum()
    data2=every_pitch[every_pitch.game_date<=end]
    data2=data2[data2.game_date>=start]
    team_abrev=team_abreviator(team,league)
    data3=data2[data2.home_team==team_abrev]
    data3=data3[data3.inning_topbot=='Bot']
    data4=data2[data2.away_team==team_abrev]
    data4=data4[data4.inning_topbot=='Top']
    data5=pd.concat([data3,data4])
    data5=data5.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
    est_ba_sa=data5.estimated_ba_using_speedangle.mean()
    est_woba_sa=data5.estimated_woba_using_speedangle.mean()
    sum_woba=data5.woba_value.sum() 
    df=pd.DataFrame([[team,league,ab,runs,hits,doub,trip,hr,rbi,bb,avg,obp,slg,
                     est_ba_sa,est_woba_sa,sum_woba]],columns=batting_columns)
    return df

In [68]:
def home_pct_chance(home_money,away_money):
    if home_money>0:
        a1=100/(home_money+100)
    else:
        a1=-home_money/(100-home_money)
    if away_money>0:
        a2=100/(away_money+100)
    else:
        a2=-away_money/(100-away_money)
    return(a1/(a1+a2))

In [20]:
pickle_in=open("cleaned_data_2021.pickle","rb")
df=pickle.load(pickle_in)

In [69]:
def convert_league(league):
    if league == "Maj-AL":
        return "MLB-AL"
    else:
        return "MLB-NL"

In [114]:
def get_matchup_stats(date, matchups, lookback_days):
    lookback_end = str(pd.to_datetime(date) - pd.to_timedelta(1, unit='D'))[:10]
    lookback_start = str(pd.to_datetime(date) - pd.to_timedelta(lookback_days + 1, unit='D'))[:10]
    starters_on_day = get_all_starters(statcast_stats, lookback_start, lookback_end)
    
#     display(pitching_stats)
    
    pitch_mask = (pitching_stats['Date'] >= lookback_start) & (pitching_stats['Date'] <= lookback_end)
    bat_mask = (batting_stats['Date'] >= lookback_start) & (batting_stats['Date'] <= lookback_end)
    
    match_pitching_stats = pitching_stats.loc[pitch_mask]
    
#     display(match_pitching_stats)
    
#     display(match_pitching_stats)
    match_batting_stats = batting_stats.loc[bat_mask]

    all_reliever_stats = get_all_relievers(starters_on_day, match_pitching_stats, lookback_start, lookback_end)

    
    data=pd.DataFrame([])
    
    for home_team, home_starter, home_money_open, home_money_close, away_team, away_starter, away_money_open, away_money_close in matchups:
        
        home_team_city, home_team_league = team_dict[home_team]
        home_abr = team_abreviator(home_team_city, convert_league(home_team_league))
        home_starter_adv=statcast_pitcher(lookback_start, lookback_end,int(home_starter))
        away_team_city, away_team_league = team_dict[away_team]
        away_abr = team_abreviator(away_team_city, convert_league(away_team_league))
        away_starter_adv=statcast_pitcher(lookback_start, lookback_end,int(away_starter))

                                          
        home_pct, home_streak = get_record_stats(date, lookback_days, home_abr)
        home_starter_launch = home_starter_adv.launch_speed.mean()
        home_relief = recent_bullpen(all_reliever_stats, statcast_stats, home_team_city, home_team_league, lookback_start, lookback_end)
        home_relief = home_relief.add_prefix('homepen_')
        home_batting = recent_team_batting(match_batting_stats, statcast_stats, lookback_start, lookback_end, home_team_city, home_team_league)
        home_batting = home_batting.add_prefix('home_bat_')
        home_starter_stats = match_pitching_stats[match_pitching_stats.mlbID == str(home_starter)]
        home_starter_stats = home_starter_stats.add_prefix('hs')
        home_prob_open = home_pct_chance(home_money_open, away_money_open)
        home_prob_close = home_pct_chance(home_money_close, away_money_close)
        
        if home_starter_stats.shape[0] == 0:
            print(home_team + " No Home Starter Stats")
            continue
        
        home_starter_stats = home_starter_stats.tail(1)
            
#         home_starter_stats['hsG'] = 1.0
#         home_starter_stats['hsGS'] = 1.0
#         if home_starter_stats['hsW'] < 1.0:
#             home_starter_stats['hsW'] = 1
     
        home_starter_stats.iloc[0] = home_starter_stats.iloc[0].apply(lambda x: 0.0 if not isinstance(x, str) and math.isnan(x) else x)
 
            
        if math.isnan(home_starter_launch):
            print(home_team + " No Home Starter Launch Stats")
            continue

        if home_starter_stats['hsW'].isna().any():
            home_starter_stats['hsW'] = 0.0
        if home_starter_stats['hsL'].isna().any():
            home_starter_stats['hsL'] = 0.0
        if home_starter_stats['hsSV'].isna().any():
            home_starter_stats['hsSV'] = 0.0
            
        away_pct, away_streak = get_record_stats(date, lookback_days, away_abr)
        away_starter_launch = away_starter_adv.launch_speed.mean()
        away_relief = recent_bullpen(all_reliever_stats, statcast_stats, away_team_city, away_team_league, lookback_start, lookback_end)
        away_relief = away_relief.add_prefix('awaypen_')
        away_batting = recent_team_batting(match_batting_stats, statcast_stats, lookback_start, lookback_end, away_team_city, away_team_league)
        away_batting = away_batting.add_prefix('away_bat_')
        away_starter_stats = match_pitching_stats[match_pitching_stats.mlbID == str(away_starter)]
        away_starter_stats = away_starter_stats.add_prefix('as')
         
        if away_starter_stats.shape[0] == 0:
            print(away_team + " No Away Starter Stats")
            continue
        
        away_starter_stats = away_starter_stats.tail(1)
            
#         away_starter_stats['asG'] = 1.0
#         away_starter_stats['asGS'] = 1.0
#         if away_starter_stats['asW'] < 1.0:
#             away_starter_stats['asW'] = 1
            
        away_starter_stats.iloc[0] = away_starter_stats.iloc[0].apply(lambda x: 0.0 if not isinstance(x, str) and math.isnan(x) else x)

        if math.isnan(away_starter_launch):
            print(away_team + " No Away Starter Launch Stats")
            continue
        
#         if math.isnan(away_starter_stats['asW']):
#             away_starter_stats['asW'] = 0.0
#         if math.isnan(away_starter_stats['asL']):
#             away_starter_stats['asL'] = 0.0
#         if math.isnan(away_starter_stats['asSV']):
#             away_starter_stats['asSV'] = 0.0

        if away_starter_stats['asW'].isna().any():
            away_starter_stats['asW'] = 0.0
        if away_starter_stats['asL'].isna().any():
            away_starter_stats['asL'] = 0.0
        if away_starter_stats['asSV'].isna().any():
            away_starter_stats['asSV'] = 0.0

        matchup_stats = pd.DataFrame({
            'home_pct': [home_pct],
            'away_pct': [away_pct],
            'home_streak': [home_streak],
            'away_streak': [away_streak],
            'home_starter_launch': [home_starter_launch],
            'away_starter_launch': [away_starter_launch],
            'home_starter_id': [str(home_starter)],
            'away_starter_id': [str(away_starter)],
            'home_team': [home_team_city],
            'home_league': [home_team_league],
            'away_team': [away_team_city],
            'away_league': [away_team_league],
            })
        matchup_stats = matchup_stats.merge(home_starter_stats, left_on='home_starter_id',
                                           right_on='hsmlbID')
        matchup_stats = matchup_stats.merge(away_starter_stats, left_on='away_starter_id',
                                            right_on='asmlbID')
        matchup_stats = matchup_stats.merge(home_batting, left_on=['home_team', 'home_league'],
                                            right_on=['home_bat_team', 'home_bat_league'])
        matchup_stats = matchup_stats.merge(away_batting, left_on=['away_team', 'away_league'],
                                            right_on=['away_bat_team', 'away_bat_league'])
        matchup_stats = matchup_stats.merge(home_relief, left_on=['home_team', 'home_league'],
                                           right_on=['homepen_team_relief', 'homepen_league_relief'])
        matchup_stats = matchup_stats.merge(away_relief, left_on=['away_team', 'away_league'],
                                           right_on=['awaypen_team_relief', 'awaypen_league_relief'])
        matchup_stats['home_money_open'] = home_money_open
        matchup_stats['home_money_change'] = home_money_close - home_money_open
        matchup_stats['away_money_open'] = away_money_open
        matchup_stats['away_money_change'] = away_money_close - away_money_open
        matchup_stats['home_prob_open'] = home_prob_open
        matchup_stats['home_prob_close'] = home_prob_close
        matchup_stats['home_prob_change'] = home_prob_close - home_prob_open
        matchup_stats['home_money_close'] = home_money_close
        matchup_stats['away_money_close'] = away_money_close
    
        cols_to_drop = list(set(matchup_stats.columns) - set(df.columns))
        matchup_stats.drop(columns=cols_to_drop, inplace=True)
        out_title = "today_matchups/" + home_team + "_" + away_team + "_" + str(date) + ".pickle"
        pickle_out=open(out_title,"wb")
        pickle.dump(matchup_stats,pickle_out)
        pickle_out.close()
        
        if not matchup_stats.empty:
            # Reset the index of the DataFrame
            matchup_stats.reset_index(drop=True, inplace=True)
            # Assign the new values to the DataFrame
                        
            matchup_stats.loc[0, "home_team"] = home_team
            matchup_stats.loc[0, "away_team"] = away_team
            matchup_stats.loc[0, "date"] = date
                        
        data = pd.concat([data, matchup_stats], ignore_index=True)
        
        
    pickle_file = "season_stats.pickle"

    # Check if the file exists
    if os.path.exists(pickle_file):
#         # Load the existing DataFrame from the pickle file
        season_stats = pd.read_pickle(pickle_file)
        
        # Append new rows to the existing DataFrame
        season_stats = season_stats.append(data, ignore_index=True)
        season_stats = season_stats.drop_duplicates(subset=['hsStS', 'home_starter_launch', 'away_bat_rbi', 'away_streak', 'home_bat_hr', 'asStL', 'home_bat_obp', 
                                                                'hsSV', 'hsCS', 'homepen_trip', 'awaypen_hr', 'away_starter_launch', 'homepen_obp', 'awaypen_bb', 
                                                                'awaypen_at_bats', 'hsBF', 'as3B', 'asStr', 'away_bat_bb', 'hsAB', 'asHBP', 'awaypen_doub', 'hsLD', 
                                                                'asBB', 'homepen_hr', 'asL', 'hsSO', 'home_bat_bb', 'hs2B', 'awaypen_obp', 'asER', 'asERA', 'asH', 'asSF', 
                                                                'hsSO9', 'asSB', 'home_bat_doub', 'asGB/FB', 'hsIP', 'hsIBB', 'hsG', 'homepen_innings', 'hsER', 'hsGDP', 
                                                                'asIP', 'hsGB/FB', 'home_bat_hits', 'asSV', 'homepen_bb', 'asBAbip', 'home_bat_ab', 'hsERA', 'awaypen_ba', 
                                                                'home_streak', 'homepen_era', 'asPit', 'asIBB', 'asWHIP', 'hsBAbip', 'asBF', 'awaypen_innings', 
                                                                'homepen_hits', 'home_bat_rbi', 'asPU', 'away_bat_runs', 'awaypen_era', 'hsPO', 'away_bat_trip', 'hsGS', 
                                                                'hs3B', 'homepen_k', 'hsStL', 'homepen_doub', 'home_bat_trip', 'asAB', 'home_bat_runs', 'hsH', 
                                                                'away_bat_hits', 'hsSO/W', 'as2B', 'asCS', 'homepen_at_bats', 'awaypen_trip', 'asG', 'homepen_slg', 
                                                                'asPO', 'hsHR', 'homepen_ba', 'hsSF', 'home_bat_avg', 'asGDP', 'hsW', 'hsPit', 'hsPU', 'awaypen_k', 
                                                                'asHR', 'hsWHIP', 'asLD', 'away_bat_obp', 'asSO9', 'hsR', 'hsStr', 'awaypen_hits', 'home_pct', 'asStS', 
                                                                'asSO', 'away_bat_hr', 'away_bat_ab', 'asR', 'hsBB', 'home_bat_slg', 'away_bat_doub', 'asGS', 'away_pct', 
                                                                'asSO/W', 'hsSB', 'hsL', 'awaypen_slg', 'asW', 'hsHBP', 'away_bat_avg', 'away_bat_slg'])
        season_stats = season_stats.reset_index(drop=True)
    
    else:
        # Create a new DataFrame with the new_data as the first rows
        season_stats = pd.DataFrame(data)
        season_stats.reset_index(drop=True, inplace=True)

        # Save the updated DataFrame back to the pickle file
#     season_stats.to_pickle(pickle_file)

    return data

In [135]:
date = '2023-04-20'
matchups_dict = {}
for idx, matchup_and_pitchers in enumerate(matchups_and_pitchers, start=1):
#     print(f"Matchup {idx}: {matchup_and_pitchers['away_team']} vs {matchup_and_pitchers['home_team']}")
#     print(f"Away pitcher: {matchup_and_pitchers['away_pitcher']}")
#     print(f"Home pitcher: {matchup_and_pitchers['home_pitcher']}")
#     print()
    matchups_dict[matchup_and_pitchers['home_team'] + "_vs_" + matchup_and_pitchers['away_team']] = [
        matchup_and_pitchers['home_team'], 
        matchup_and_pitchers['home_pitcher'],
        matchup_and_pitchers['away_team'],
        matchup_and_pitchers['away_pitcher']
    ]
# first_home_odds, last_home_odds, first_away_odds, last_away_odds = get_first_last_odds("Twins_vs_Wte Sox", "2023-04-10")
matchups = []
for key in matchups_dict.keys():
    print(key)
    first_home_odds, last_home_odds, first_away_odds, last_away_odds = get_first_last_odds(str(key), date)
    if first_home_odds == 0 and last_home_odds == 0 and first_away_odds == 0 and last_away_odds == 0:
        continue
    matchups.append((
        matchups_dict[key][0], 
        matchups_dict[key][1], 
        first_home_odds, 
        last_home_odds, 
        matchups_dict[key][2], 
        matchups_dict[key][3],
        first_away_odds,
        last_away_odds
    ))
today_matchups = get_matchup_stats(date, matchups, 8)


Red Sox_vs_Twins
Yankees_vs_Angels
Pirates_vs_Reds
Phillies_vs_Rockies
Cubs_vs_Dodgers
Diamondbacks_vs_Padres
Error: The matchup key does not exist in the data file.
Giants_vs_Mets
Error: The matchup key does not exist in the data file.
Gathering Player Data
Gathering Player Data
http://www.baseball-reference.com/teams/BOS/2023-schedule-scores.shtml
http://www.baseball-reference.com/teams/MIN/2023-schedule-scores.shtml
Twins No Away Starter Stats
Gathering Player Data
Gathering Player Data
http://www.baseball-reference.com/teams/NYY/2023-schedule-scores.shtml
http://www.baseball-reference.com/teams/LAA/2023-schedule-scores.shtml
Gathering Player Data
Gathering Player Data
http://www.baseball-reference.com/teams/PIT/2023-schedule-scores.shtml
http://www.baseball-reference.com/teams/CIN/2023-schedule-scores.shtml
Reds No Away Starter Stats
Gathering Player Data
Gathering Player Data
http://www.baseball-reference.com/teams/PHI/2023-schedule-scores.shtml
http://www.baseball-reference.com/t

In [83]:
pickle_file = "season_stats.pickle"
df = pd.read_pickle(pickle_file)
print(df.shape[0])
display(df)

48


,home_pct,away_pct,home_streak,away_streak,home_starter_launch,away_starter_launch,hsG,hsGS,hsW,hsL,hsSV,hsIP,hsH,hsR,hsER,hsBB,hsSO,hsHR,hsHBP,hsERA,hsAB,hs2B,hs3B,hsIBB,hsGDP,hsSF,hsSB,hsCS,hsPO,hsBF,hsPit,hsStr,hsStL,hsStS,hsGB/FB,hsLD,hsPU,hsWHIP,hsBAbip,hsSO9,hsSO/W,asG,asGS,asW,asL,asSV,asIP,asH,asR,asER,asBB,asSO,asHR,asHBP,asERA,asAB,as2B,as3B,asIBB,asGDP,asSF,asSB,asCS,asPO,asBF,asPit,asStr,asStL,asStS,asGB/FB,asLD,asPU,asWHIP,asBAbip,asSO9,asSO/W,home_bat_ab,home_bat_runs,home_bat_hits,home_bat_doub,home_bat_trip,home_bat_hr,home_bat_rbi,home_bat_bb,home_bat_avg,home_bat_obp,home_bat_slg,away_bat_ab,away_bat_runs,away_bat_hits,away_bat_doub,away_bat_trip,away_bat_hr,away_bat_rbi,away_bat_bb,away_bat_avg,away_bat_obp,away_bat_slg,homepen_innings,homepen_hits,homepen_bb,homepen_k,homepen_at_bats,homepen_doub,homepen_trip,homepen_hr,homepen_era,homepen_ba,homepen_slg,homepen_obp,awaypen_innings,awaypen_hits,awaypen_bb,awaypen_k,awaypen_at_bats,awaypen_doub,awaypen_trip,awaypen_hr,awaypen_era,awaypen_ba,awaypen_slg,awaypen_obp,home_money_open,home_money_change,away_money_open,away_money_change,home_prob_open,home_prob_close,home_prob_change,home_money_close,away_money_close,home_team,away_team,date
0,0.500000,0.200000,0.571429,0.142857,77.900000,80.909375,1,1,0.0,0.0,0.0,5.0,4,2,2,1,5,0,0,3.60,19,1,0,0,0,0,0,0,0,20,83,0.64,0.11,0.14,0.57,0.43,0.00,1.000,0.286,9.0,5.00,2,2,0.0,0.0,0.0,10.2,8,3,3,4,8,0,1,2.53,40,0,0,1,1,0,1,0,0,45,167,0.63,0.13,0.10,0.59,0.19,0.00,1.125,0.250,6.8,2.00,331,50,78,20,0,15,48,45,0.236,0.372,0.432,323,28,64,15,1,8,27,33,0.198,0.300,0.325,87.666667,8.828897,3.182510,8.726236,34.494297,1.745247,0.410646,1.334601,4.825095,0.255952,0.446,0.319,88.000000,9.818182,6.238636,7.056818,35.590909,1.431818,0.204545,2.045455,7.363636,0.275862,0.500,0.384,-153.846154,-34.833091,135.000000,25.000000,0.587500,0.629540,0.042040,-188.679245,160.000000,Orioles,Athletics,2023-04-11
1,0.600000,0.454545,0.714286,0.428571,78.427500,77.983784,2,2,1.0,0.0,0.0,11.2,10,5,5,6,15,1,0,3.86,43,1,1,0,1,0,1,0,0,49,208,0.68,0.20,0.11,0.57,0.18,0.11,1.371,0.333,11.6,2.50,2,2,1.0,0.0,0.0,11.0,13,4,4,1,11,0,0,3.27,45,4,0,0,1,0,6,0,0,46,181,0.74,0.17,0.15,0.38,0.24,0.06,1.273,0.382,9.0,11.00,328,44,76,17,2,12,42,40,0.232,0.354,0.405,380,55,95,15,0,10,53,52,0.250,0.387,0.368,88.000000,8.795455,4.397727,8.284091,34.977273,1.738636,0.102273,1.329545,4.909091,0.251462,0.421,0.335,100.333333,9.059801,3.588040,9.418605,34.983389,1.794020,0.089701,0.986711,3.767442,0.258974,0.400,0.328,150.000000,-5.000000,-175.438596,5.947071,0.385749,0.393564,0.007815,145.000000,-169.491525,Pirates,Astros,2023-04-11
2,0.400000,0.363636,0.571429,0.428571,82.096667,81.114286,2,2,0.0,1.0,0.0,9.2,11,8,8,3,9,1,0,7.45,38,3,0,0,1,0,2,1,0,41,159,0.69,0.23,0.08,0.34,0.34,0.07,1.448,0.357,8.4,3.00,2,2,1.0,0.0,0.0,12.2,7,1,1,5,15,0,0,0.71,43,1,0,0,1,0,2,2,0,48,190,0.64,0.18,0.17,0.46,0.11,0.14,0.947,0.250,10.7,3.00,347,45,99,24,5,10,43,27,0.285,0.363,0.470,352,30,82,10,1,12,29,33,0.233,0.327,0.369,85.000000,9.105882,5.082353,8.894118,34.623529,1.482353,0.105882,1.270588,5.400000,0.262997,0.422,0.357,96.000000,8.531250,4.781250,7.687500,34.687500,1.968750,0.093750,1.406250,5.531250,0.245946,0.430,0.337,-153.846154,13.001083,135.000000,-15.000000,0.587500,0.562660,-0.024840,-140.845070,120.000000,Phillies,Marlins,2023-04-11
3,0.600000,0.222222,0.625000,0.285714,84.809091,81.706250,2,2,1.0,0.0,0.0,10.1,10,5,5,6,8,2,1,4.35,40,0,0,0,0,0,0,0,0,47,183,0.63,0.16,0.12,0.38,0.25,0.03,1.548,0.267,7.0,1.33,1,1,1.0,0.0,0.0,5.2,6,2,2,2,4,1,0,3.18,21,0,0,0,0,0,0,0,1,23,89,0.61,0.17,0.07,0.41,0.24,0.00,1.412,0.313,6.4,2.00,368,54,106,17,1,10,50,39,0.288,0.394,0.421,308,27,63,10,1,5,26,24,0.205,0.282,0.292,87.000000,9.413793,4.034483,8.689655,35.275862,1.344828,0.103448,1.758621,5.275862,0.266862,0.460,0.342,79.000000,9.455696,5.012658,6.265823,35.202532,2.392405,0.000000,1.822785,6.835443,0.268608,0.492,0.360,-263.157895,-14.619883,225.000000,5.000000,0.701944,0.708155,0.006211,-

In [126]:
# pickle_in=open("cleaned_data_2021.pickle","rb")
# df=pickle.load(pickle_in)
# cols_to_drop = list(set(today_matchups.columns) - set(df.columns))
# today_matchups.drop(columns=cols_to_drop, inplace=True)
# # display(today_matchups)
# pd.set_option('display.max_rows', 2000)
display(today_matchups)
display(today_matchups.mean())

,home_pct,away_pct,home_streak,away_streak,home_starter_launch,away_starter_launch,hsG,hsGS,hsW,hsL,hsSV,hsIP,hsH,hsR,hsER,hsBB,hsSO,hsHR,hsHBP,hsERA,hsAB,hs2B,hs3B,hsIBB,hsGDP,hsSF,hsSB,hsCS,hsPO,hsBF,hsPit,hsStr,hsStL,hsStS,hsGB/FB,hsLD,hsPU,hsWHIP,hsBAbip,hsSO9,hsSO/W,asG,asGS,asW,asL,asSV,asIP,asH,asR,asER,asBB,asSO,asHR,asHBP,asERA,asAB,as2B,as3B,asIBB,asGDP,asSF,asSB,asCS,asPO,asBF,asPit,asStr,asStL,asStS,asGB/FB,asLD,asPU,asWHIP,asBAbip,asSO9,asSO/W,home_bat_ab,home_bat_runs,home_bat_hits,home_bat_doub,home_bat_trip,home_bat_hr,home_bat_rbi,home_bat_bb,home_bat_avg,home_bat_obp,home_bat_slg,away_bat_ab,away_bat_runs,away_bat_hits,away_bat_doub,away_bat_trip,away_bat_hr,away_bat_rbi,away_bat_bb,away_bat_avg,away_bat_obp,away_bat_slg,homepen_innings,homepen_hits,homepen_bb,homepen_k,homepen_at_bats,homepen_doub,homepen_trip,homepen_hr,homepen_era,homepen_ba,homepen_slg,homepen_obp,awaypen_innings,awaypen_hits,awaypen_bb,awaypen_k,awaypen_at_bats,awaypen_doub,awaypen_trip,awaypen_hr,awaypen_era,awaypen_ba,awaypen_slg,awaypen_obp,home_money_open,home_money_change,away_money_open,away_money_change,home_prob_open,home_prob_close,home_prob_change,home_money_close,away_money_close,home_team,away_team,date
0,0.357143,0.562500,0.428571,0.571429,79.988235,87.019048,1,1,0.0,0.0,0.0,6.0,7,1,1,1,3,0,0,1.50,22,1,0,0,1,0,0,0,0,23,96,0.67,0.19,0.07,0.47,0.37,0.05,1.333,0.368,4.5,3.0,1,1,0.0,0.0,0.0,4.2,4,2,1,0,3,0,0,1.93,18,1,0,0,0,0,0,0,0,18,66,0.68,0.11,0.23,0.47,0.20,0.00,0.857,0.267,5.8,0.00,214,24,47,16,1,5,24,23,0.220,0.327,0.374,237,31,53,18,2,3,28,43,0.224,0.405,0.354,57.333333,8.476744,3.296512,8.947674,34.063953,1.098837,0.000000,1.569767,4.238372,0.248848,0.419,0.315,65.000000,9.276923,3.184615,7.061538,35.169231,1.800000,0.138462,0.830769,4.292308,0.263780,0.394,0.325,-140.845070,250.84507,110.000000,-239.87013,0.551181,0.457364,-0.093817,110.000000,-129.870130,Tigers,Guardians,2023-04-18
1,0.375000,0.375000,0.285714,0.375000,84.027778,82.589189,1,1,0.0,1.0,0.0,6.0,5,1,1,2,7,0,1,1.50,23,0,1,0,0,0,0,0,0,26,108,0.60,0.14,0.11,0.25,0.19,0.06,1.167,0.313,10.5,3.5,1,1,0.0,1.0,0.0,4.2,8,5,5,1,4,0,0,9.64,21,1,0,0,0,0,1,1,1,22,94,0.68,0.14,0.09,0.71,0.18,0.06,1.929,0.471,7.7,4.00,236,22,50,12,0,7,22,17,0.212,0.284,0.352,300,49,96,23,2,9,48,30,0.320,0.420,0.500,62.000000,7.983871,5.951613,10.161290,34.112903,1.306452,0.580645,0.580645,3.629032,0.234043,0.357,0.348,71.000000,9.507042,5.070423,9.380282,35.366197,2.154930,0.253521,0.760563,5.323944,0.268817,0.409,0.361,105.000000,-234.87013,-125.000000,235.00000,0.467532,0.542636,0.075103,-129.870130,110.000000,White Sox,Phillies,2023-04-18
2,0.437500,0.823529,0.375000,0.625000,83.396296,83.064000,1,1,1.0,0.0,0.0,5.0,8,2,2,2,6,1,0,3.60,22,1,0,0,1,0,0,0,0,24,97,0.64,0.19,0.14,0.50,0.25,0.00,2.000,0.467,10.8,3.0,1,1,1.0,0.0,0.0,5.0,5,3,3,1,8,0,0,5.40,19,2,0,0,1,0,0,0,0,20,78,0.67,0.23,0.12,0.36,0.36,0.09,1.200,0.455,14.4,8.00,304,53,77,18,1,4,51,46,0.253,0.405,0.359,309,51,84,20,0,16,50,32,0.272,0.375,0.492,79.000000,10.822785,4.329114,8.772152,36.455696,2.506329,0.227848,1.139241,5.126582,0.296875,0.472,0.372,78.000000,6.230769,4.384615,8.884615,32.653846,1.846154,0.000000,0.461538,3.576923,0.190813,0.290,0.287,110.000000,0.00000,-129.870130,0.00000,0.457364,0.457364,0.000000,110.000000,-129.870130,Reds,Rays,2023-04-18
3,0.529412,0.333333,0.714286,0.166667,83.011111,87.077778,1,1,0.0,0.0,0.0,5.0,7,2,2,1,4,0,0,3.60,21,3,0,0,1,0,2,0,0,22,82,0.61,0.17,0.10,0.65,0.12,0.00,1.600,0.412,7.2,4.0,1,1,0.0,0.0,0.0,4.2,1,0,0,3,5,0,2,0.00,15,0,0,0,0,0,0,0,0,20,75,0.56,0.16,0.07,0.30,0.00,0.00,0.857,0.100,9.6,1.67,272,33,75,12,2,8,32,22,0.276,0.357,0.423,245,28,56,9,0,9,28,26,0.229,0.335,0.376,72.000000,9.375000,2.375000,9.375000,35.250000,2.750000,0.000000,0.625000,4.000000,0.265957,0.397,0.312,64.666667,8.211340,3.340206,9.603093,34.237113,1.530928,0.417526,1.252577,4.175258,0.239837,0.419,0.307,-114.942529,0.00000,-105.263158,0.00000,0.510471,0.510471,0.000000,-114.942529,-105.263158,Marlins,Giants

home_pct                 0.417134
away_pct                 0.555011
home_streak              0.414683
away_streak              0.508598
home_starter_launch     82.296730
away_starter_launch     83.394349
hsG                      1.000000
hsGS                     1.000000
hsW                      0.222222
hsL                      0.444444
hsSV                     0.000000
hsIP                     5.277778
hsH                      5.888889
hsR                      2.222222
hsER                     2.111111
hsBB                     1.555556
hsSO                     4.888889
hsHR                     0.555556
hsHBP                    0.444444
hsERA                    3.952222
hsAB                    21.111111
hs2B                     1.444444
hs3B                     0.111111
hsIBB                    0.000000
hsGDP                    0.666667
hsSF                     0.222222
hsSB                     0.333333
hsCS                     0.000000
hsPO                     0.000000
hsBF          